# Quick introduction


erddapy is a pure python package and can be installed with conda


```shell
conda install --channel conda-forge erddapy
```

 or pip

```shell
pip install erddapy
```

First we need to instantiate the ERDDAP URL constructor for a server.
In this example we will use [https://gliders.ioos.us/erddap](https://gliders.ioos.us/erddap/index.html).

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
    server="NGDAC", # "NOAA IOOS NGDAC (National Glider Data Assembly Center)"
    protocol="tabledap",
    response="csv",
)

Now we can populate the object a dataset id, variables of interest, and 
its constraints. We can download the csvp response with the `.to_pandas` method.

In [ ]:
e.dataset_id = "whoi_406-20160902T1700"

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "time",
]

e.constraints = {
    "time>=": "2016-09-03T00:00:00Z",
    "time<=": "2016-09-04T00:00:00Z",
    "latitude>=": 38.0,
    "latitude<=": 41.0,
    "longitude>=": -72.0,
    "longitude<=": -69.0,
}


df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
).dropna()

df.head()